# Créer un regexp pattern pour Google Analytics en scrapant des liens

Pour trouver les stats d’un type de contenu qui n’est pas balisé et hébergé de manière homogène.

A utiliser dans la recherche «Toutes les pages» avec «Titre de page» comme primary dimension.

Amusez-vous bien!

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
# On récupère les liens et dates - à adapter selon la page
response = requests.get('https://www.tdg.ch/grandformat/')
doc = BeautifulSoup(response.content, 'html.parser')

anchors = []
for a, t in zip(doc.select('h3 a'), doc.select('.time')):
    anchors.append((a.string.strip(), a.get('href'), t.string))

In [6]:
# Ajout manuel de liens manquants
anchors.extend([
    ('Les milliards du peuple angolais font la fortune d’un entrepreneur suisse', 'https://www.tdg.ch/extern/interactive_wch/paradise-papers-tdg/angola-bastos/', '05.11.2017'),
    ('Les affaires opaques de sociétés suisses en Afrique', 'https://www.tdg.ch/extern/interactive_wch/paradise-papers-tdg/', '05.11.2017'),
    ('Au cœur des méandres fiscaux de Nike', 'https://www.tdg.ch/extern/interactive_wch/paradise-papers-tdg/inside-nike/', '05.11.2017'),
    ('Bientôt un congé paternité de deux semaines pour tous?', 'http://paternite.tdg.ch/', '01.03.2015')
])

In [7]:
# On en fait un dataframe
df = pd.DataFrame(anchors, columns = ['Titre', 'URL', 'Date'])

In [8]:
# Tri par nom de domaine
def get_host(url):
    if url[-7:] == 'tdg.ch/' or url[-16:] == 'tdg.ch/index.php' or url[-6:] == 'tdg.ch':
        return 'Cortex'
    elif url.find('longforms.tdg') >= 0:
        return 'OVH'
    elif url.find('extern') >= 0:
        return 'Zurich'
    else:
        return 'Autre'
df['URL'].apply(get_host).value_counts()

Cortex    14
Zurich     8
OVH        4
Autre      3
Name: URL, dtype: int64

In [9]:
df['Hôte'] = df['URL'].apply(get_host)

# On refait le dataframe sans hôte «autre»
df = df[df['Hôte'] != 'Autre'].copy()
df.reset_index(inplace=True)

In [10]:
# Cette colonne contiendra le <title> des pages
df['Pagetitle'] = ''

In [74]:
for i, row in df[16:17].iterrows():
    print(row['URL'], end='… ')
    response = requests.get(row['URL'])
    doc = BeautifulSoup(response.content, 'html.parser')
    title = doc.find('title')
    if title:
        df.at[i, 'Pagetitle'] = title.string
        print(title.string)

http://jeuvideo.tdg.ch/… Jeu vidéo: six créateurs dévoilent leur univers – Tribune de Genève


In [59]:
# Remaniement manuel
df.at[15, 'Pagetitle'] = 'Genève, ville d’espions'
df.at[14, 'Pagetitle'] = 'Paradise Papers'

Titre        Comment Genève est devenu Suisse
URL               http://bicentenaire.tdg.ch/
Date                               19.05.2015
Hôte                                   Cortex
Pagetitle    Comment Genève est devenu suisse
Name: 23, dtype: object

In [82]:
df

,Titre,URL,Date,Hôte,Pagetitle
0,Quand les femmes passent à l’action,https://www.tdg.ch/extern/interactive_wch/tdg/...,13.11.2017,Zurich,Quand les femmes passent à l’action
1,Toutes nos enquêtes Paradise papers,https://www.tdg.ch/extern/interactive_wch/para...,09.11.2017,Zurich,Paradise Papers
2,La vie dorée des fils d'un potentat africain à...,https://www.tdg.ch/extern/interactive_wch/tdg/...,27.10.2017,Zurich,Les trophées genevois des fils du président
3,Servette succombe à la fièvre du e-sport,https://www.tdg.ch/extern/interactive_wch/tdg/...,26.09.2017,Zurich,"Servette, la fièvre du e-sport – Grand format"
4,Course du Duc: vos plans d’entraînement person...,https://www.tdg.ch/extern/interactive_wch/tdg/...,08.09.2017,Zurich,Course du Duc: vos plans d’entraînement person...
5,La 8e quête de Federer,http://longforms.tdg.ch/federer_wimbledon/,29.06.2017,OVH,La 8e quête – Grand format
6,Quinze projets pour un monde meilleur,http://longforms.tdg.ch/ijd2017/,24.06.2017,OVH,Quinze projets pour un monde meilleur - Impact...
7,Les grandes gueules du Grand Conseil,http://longforms.tdg.ch/grandconseil/,01.06.2017,OVH,Les grandes gueules du Grand Conseil
8,"Di-Meh, Slimka, Makala, le rap qui enflamme Ge...",http://longforms.tdg.ch/rap,26.05.2017,OVH,"Makala, Di-Meh, Slimka, le rap qui enflamme Ge..."
9,Course de l’Escalade: les chiffres d’un succès...,http://escalade.tdg.ch/index.php,28.11.2016,Cortex,Course de l’Escalade: les chiffres d’un succès...


In [76]:
df.to_csv('longform_pagetitles.csv')

In [83]:
# Création du pattern pour Google Analytics
titles = df['Pagetitle'].values
print("|".join( ["(^{}$)".format(i.replace('?', '\?')) for i in titles] ))

(^Quand les femmes passent à l’action$)|(^Paradise Papers$)|(^Les trophées genevois des fils du président$)|(^Servette, la fièvre du e-sport – Grand format$)|(^Course du Duc: vos plans d’entraînement personnalisés$)|(^La 8e quête – Grand format$)|(^Quinze projets pour un monde meilleur - Impact Journalism Day 2017$)|(^Les grandes gueules du Grand Conseil$)|(^Makala, Di-Meh, Slimka, le rap qui enflamme Genève – Grands formats$)|(^Course de l’Escalade: les chiffres d’un succès populaire$)|(^Impact Journalism Day – Tribune de Genève$)|(^Se mettre au monde - Steeve Iuncker$)|(^Dans la foulée de Tadesse Abraham$)|(^A qui appartient Genève - Introduction$)|(^Paradise Papers$)|(^Genève, ville d’espions$)|(^Jeu vidéo: six créateurs dévoilent leur univers – Tribune de Genève$)|(^Meurtre aux promotions$)|(^Cervin jubilée des 150ans de la première ascension$)|(^Impact Journalism Day – Tribune de Genève$)|(^Comment Genève est devenu suisse$)|(^La Tribune de Genève à l'ère du numérique$)|(^Les mill

# Et voilà!
Les `^` et `$` (marquage de début et fin) permettent une recherche exacte pour éviter qu’un titre court ramène des résultats indésirables.

Scripté par Paul Ronga pour la [Tribune de Genève](https://www.tdg.ch), licence [GPLv3](https://www.gnu.org/licenses/gpl-3.0.html).

